In [55]:
import sys
import pandas as pd
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_score,completeness_score,v_measure_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

from sklearn.feature_selection import RFE

TRAIN_TEST_RATIO = 0.25

In [56]:

data = pd.read_csv('dermatology.data.txt', sep=",", header = None)

# [1-11,34] podaci pre analiza
# [12-33] podaci nakon analiza
data.columns = cols = ["erythema", "scaling", "definite borders", "itching","koebner phenomenon",
                "polygonal papules","follicular papules","oral mucosal involvement","knee and elbow involvement",
                "scalp involvement","family history","melanin incontinence","eosinophils in the infiltrate","PNL infiltrate",
                "fibrosis of the papillary dermis","exocytosis","acanthosis","hyperkeratosis","parakeratosis","clubbing of the rete ridges",
               "elongation of the rete ridges","thinning of the suprapapillary epidermis","spongiform pustule","munro microabcess","focal hypergranulosis",
               "disappearance of the granular layer","vacuolisation and damage of basal layer","spongiosis","saw-tooth appearance of retes","follicular horn plug",
               "perifollicular parakeratosis","inflammatory monoluclear inflitrate","band-like infiltrate","Age","des_class"]

# data.head()

#izbaciti redove koje sadrze '?' u koloni za godine
# print(len(data))
data = data[data.Age != '?']
print(len(data))
data = data[data.des_class != 6]
print(len(data))

#izbaciti godine za probu

# data = data.drop('Age',1)


results = data['des_class']

#normalize




cols = data.columns

min_max_scaler = MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)


data.columns = cols

attributes = data.drop('des_class',1)
results.head()
# print(attributes[attributes.columns[1]])
# attributes.head()
# print(len(results))

358
338


0    2
1    1
2    3
3    1
4    3
Name: des_class, dtype: int64

In [57]:

train_data, test_data,train_results,test_results = train_test_split(attributes, results ,test_size=TRAIN_TEST_RATIO)


print('%d - %d' % (len(train_results), len(test_results)))


253 - 85


In [58]:

# naci najbitinije feature
model = ExtraTreesClassifier(n_estimators=250)
model.fit(data, results)
print(model.feature_importances_)
# cols = deepcopy(train_data.columns)
# print(len(cols))
# for idx,elem in enumerate(model.feature_importances_):
    
#     if float(elem) < float(0.01):
#         print(cols[idx])
       
#         train_data = train_data.drop(cols[idx],1)
#         test_data = test_data.drop(cols[idx],1)


model = LogisticRegression()
rfe = RFE(model, 10)
fit = rfe.fit(data, results)
print(fit.n_features_)
print(fit.support_)
print(fit.ranking_)

cols = deepcopy(data.columns)

print(len(cols))

for idx,elem in enumerate(fit.support_):
    
    if not elem:
        print(elem)
        print(idx)
        print(cols[idx])
       
        train_data = train_data.drop(cols[idx],1)
        test_data = test_data.drop(cols[idx],1)




[  4.94728349e-03   9.87368852e-03   1.24012830e-02   1.60621677e-02
   3.04420837e-02   3.58082181e-02   1.92818226e-03   2.51674646e-02
   2.48761139e-02   2.61917143e-02   1.85057162e-03   3.53796732e-02
   7.22383763e-03   2.59601553e-02   5.30088680e-02   3.03323679e-02
   4.74552603e-03   3.31876880e-03   6.89644939e-03   6.41506867e-02
   6.48097325e-02   6.19069677e-02   5.44049191e-03   1.60615296e-02
   3.28794795e-02   1.87504955e-02   3.80581240e-02   5.11317374e-02
   4.53365311e-02   2.22628475e-04   6.79142678e-05   3.32338010e-03
   4.02352902e-02   2.47697679e-03   1.98733617e-01]
10
[False False False False  True  True False False False False False False
 False False  True False False False False  True  True  True False False
 False False False  True  True False False False  True False  True]
[16  5 14  2  1  1 21  8 10 12 24  9 13  3  1  6 15 23 22  1  1  1 19 17 11
  7  4  1  1 25 26 18  1 20  1]
35
False
0
erythema
False
1
scaling
False
2
definite borders
False
3
i

In [59]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_data,train_results)


In [60]:
predicted = kmeans.predict(test_data)


print(homogeneity_score(predicted,test_results))
print(completeness_score(predicted,test_results))
print(v_measure_score(predicted,test_results))


0.895434959676
0.845977883258
0.870004117467
